In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!mkdir -p /content/drive/MyDrive/gpt2-lyrics


In [ ]:
!pip install transformers datasets


In [ ]:
import os

# Folder with all the lyrics
lyrics_dir = "/content/drive/MyDrive/gpt2-lyrics"

# Combine all lyrics into one file
combined_path = "/content/lyrics.txt"

with open(combined_path, "w", encoding='utf-8') as outfile:
    for filename in os.listdir(lyrics_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(lyrics_dir, filename), "r", encoding='utf-8') as infile:
                outfile.write(infile.read() + "\n\n")


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

dataset = load_dataset(combined_path, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/gpt2-lyrics-model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)


In [ ]:
trainer.train()


In [ ]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
set_seed(42)

generated_lyrics = generator("Love is", max_length=100, num_return_sequences=1)
print(generated_lyrics[0]['generated_text'])


In [ ]:
https://colab.research.google.com/drive/1uwi0LWiJYd3W2H0krDb61QSHGLpYIdyQ?usp=sharing